In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 2)

os.chdir("/home/coinse/Workspace/SBFL")

# Load bug info

In [2]:
from scripts.util import load_bug_data, find_covmat_func, find_covmat_idx
from functools import reduce
from typing import Set

pid, vid = "proj", 11

bug = load_bug_data(pid, vid)

print(f"{bug.buggy_files=}")
print(f"{bug.buggy_lines=}")
print("bug.coverage_df=")
display(bug.coverage_df)
print(f"{bug.failing_tests=}")
print(f"{bug.failing_tests_GT=}")
print("bug.functions={", end="")
for func in list(bug.functions)[:5]:
    print(f"{func},", end=" ")
print(", ...}")
print(f"{bug.id=}")
print(f"{bug.num_test_cases=}")
print(f"{bug.passing_tests=}")
print(f"{bug.path_to_cov_df=}")
print(f"{bug.path_to_patch=}")
print(f"{bug.path_to_test_results=}")
print("bug.source_files=", end="")
for src in list(bug.source_files)[:5]:
    print(f"{src},", end=" ")
print(", ...}")

fault_idxs = sorted(bug.buggy_lines)
# fault_idx2meth = {}
fault_meth2idx = {}
err_msg = ""
for fault_idx in fault_idxs:
    succ_func, ret_func = find_covmat_func(
        pid, bug.coverage_df, fault_idx, debug=False
    )
    if succ_func:
        succ_idx, ret_idx = find_covmat_idx(
            pid, bug.coverage_df, fault_idx, debug=False
        )
        if succ_idx:
            if ret_func in fault_meth2idx:
                fault_meth2idx[ret_func].add(ret_idx)
            else:
                fault_meth2idx[ret_func] = {ret_idx}
        else:
            print(f"{ret_idx}")
            if ret_func not in fault_meth2idx:
                fault_meth2idx[ret_func] = set()
    else:
        print(f"{ret_func}")
# fault_meths: Set = reduce(
#     lambda x, y: x | set([y]), fault_idx2meth.values(), set()
# )
# fault_meth2idx = {
#     v: {idx for idx in fault_idx2meth if fault_idx2meth[idx] == v}
#     for v in fault_meths
# }
fault_meths = set(fault_meth2idx.keys())
for meth in fault_meth2idx:
    print(f"{meth=}: {fault_meth2idx[meth]}")
# print(f"{fault_meths=}")


bug.buggy_files={'src/projections/lcc.cpp'}
bug.buggy_lines={('src/projections/lcc.cpp', 111), ('src/projections/lcc.cpp', 110)}
bug.coverage_df=


16  \
file                                      function                                                                                             line       
/usr/include/c++/9/initializer_list       NaN                                                                                                  71    0    
                                          _ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE4sizeEv                      71    0    
                                          _ZNKSt16initializer_listIN7dropbox6oxygen2nnISt10shared_ptrIN5osgeo4proj5datum9EllipsoidEEEEE4sizeEv 71    0    
                                                                                                                                               75    0    
                                          _ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE5beginEv                     75    0    
...                                                                                                                                                 ..    
/home/workspace/src/projections/putp6.cpp pj_putp6                                                                                             91    0    
                                                                                                                                               93    0    
                                                                                                                                               94    0    
                                                                                                                                               95    0    
                                                                                                                                               97    0    

                                                                                                                                                     11  \
file                                      function                                                                                             line       
/usr/include/c++/9/initializer_list       NaN                                                                                                  71    0    
                                          _ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE4sizeEv                      71    0    
                                          _ZNKSt16initializer_listIN7dropbox6oxygen2nnISt10shared_ptrIN5osgeo4proj5datum9EllipsoidEEEEE4sizeEv 71    0    
                                                                                                                                               75    0    
                                          _ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE5beginEv                     75    0    
...                                                                                                                                                 ..    
/home/workspace/src/projections/putp6.cpp pj_putp6                                                                                             91    0    
                                                                                                                                               93    0    
                                                                                                                                               94    0    
                                                                                                                                               95    0    
                                                                                                                                               97    0    

                                                                                                                     

bug.failing_tests={'2'}
bug.failing_tests_GT={'2'}
bug.functions={('/home/workspace/src/projections/stere.cpp', '_ZL5setupP8PJconsts'), ('/usr/include/c++/9/tuple', '_ZNSt11_Tuple_implILm0EJPN5osgeo4proj3crs10DerivedCRS7PrivateESt14default_deleteIS4_EEEC2Ev'), ('/home/workspace/src/projections/aitoff.cpp', '_Z35pj_projection_specific_setup_aitoffP8PJconsts'), ('/usr/include/c++/9/bits/unordered_map.h', '_ZNSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEESt14_List_iteratorIN5osgeo4proj5lru1112KeyValuePairIS5_St6vectorIN7dropbox6oxygen2nnISt10shared_ptrINS8_9operation19CoordinateOperationEEEESaISJ_EEEEESt4hashIS5_ESt8equal_toIS5_ESaISt4pairIKS5_SN_EEE5eraseERST_'), ('/home/workspace/src/projections/nsper.cpp', '_ZL5setupP8PJconsts'), , ...}
bug.id='proj-11'
bug.num_test_cases=39
bug.passing_tests={'13', '21', '8', '15', '19', '17', '37', '25', '38', '22', '32', '6', '5', '24', '33', '14', '10', '20', '11', '31', '9', '18', '39', '12', '23', '26', '34', '3', '35', '2

# Get spectra

In [3]:
covmat = bug.coverage_df
pass_tcs = bug.passing_tests
fail_tcs = bug.failing_tests

covmat_bool = covmat.astype(bool)
P = len(pass_tcs)
F = len(fail_tcs)
e_p = covmat_bool[pass_tcs].sum(axis=1)
e_f = covmat_bool[fail_tcs].sum(axis=1)
n_p = P - e_p
n_f = F - e_f
spectra = pd.DataFrame(
    {
        "line": covmat.index.to_list(),
        "e_p": e_p,
        "e_f": e_f,
        "n_p": n_p,
        "n_f": n_f,
    }
)

display(spectra)


line  \
file                                      function                                                                                             line                                                                                                                                                    
/usr/include/c++/9/initializer_list       NaN                                                                                                  71    (/usr/include/c++/9/initializer_list, nan, 71)                                                                                                    
                                          _ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE4sizeEv                      71    (/usr/include/c++/9/initializer_list, _ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE4sizeEv, 71)                        
                                          _ZNKSt16initializer_listIN7dropbox6oxygen2nnISt10shared_ptrIN5osgeo4proj5datum9EllipsoidEEEEE4sizeEv 71    (/usr/include/c++/9/initializer_list, _ZNKSt16initializer_listIN7dropbox6oxygen2nnISt10shared_ptrIN5osgeo4proj5datum9EllipsoidEEEEE4sizeEv, 71)   
                                                                                                                                               75    (/usr/include/c++/9/initializer_list, _ZNKSt16initializer_listIN7dropbox6oxygen2nnISt10shared_ptrIN5osgeo4proj5datum9EllipsoidEEEEE4sizeEv, 75)   
                                          _ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE5beginEv                     75    (/usr/include/c++/9/initializer_list, _ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE5beginEv, 75)                       
...                                                                                                                                                                                                                                                                                              ...   
/home/workspace/src/projections/putp6.cpp pj_putp6                                                                                             91    (/home/workspace/src/projections/putp6.cpp, pj_putp6, 91)                                                                                         
                                                                                                                                               93    (/home/workspace/src/projections/putp6.cpp, pj_putp6, 93)                                                                                         
                                                                                                                                               94    (/home/workspace/src/projections/putp6.cpp, pj_putp6, 94)                                                                                         
                                                                                                                                               95    (/home/workspace/src/projections/putp6.cpp, pj_putp6, 95)                                                                                         
                                                                                                                                               97    (/home/workspace/src/projections/putp6.cpp, pj_putp6, 97)                                                                                         

                                                                                                                                                     e_p  \
file                                      function                                                                                             line        
/usr/include/c++/9/initializer_list       NaN                                                                                  

# Compute suspiciousness scores

In [4]:
def ochiai(e_f, e_p, n_f, n_p):
    return e_f / np.sqrt(e_p + e_f) / np.sqrt(e_f + n_f) if e_f else 0.0


def op2(e_f, e_p, n_f, n_p):
    return e_f - e_p / (e_p + n_p + 1)

spectra["Sop2"] = spectra.apply(
    lambda row: op2(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Sochiai"] = spectra.apply(
    lambda row: ochiai(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Rop2"] = spectra.Sop2.rank(ascending=False, method="max")
spectra["Rochiai"] = spectra.Sochiai.rank(ascending=False, method="max")


# Line level SBFL

In [5]:
for fault_meth in fault_meth2idx:
    fault_idxs_of_meth = fault_meth2idx[fault_meth]
    print("fault meth:", fault_meth)
    for fault_idx in fault_idxs_of_meth:
        print("fault idx:", tuple(fault_idx))
    print("SBFL score of faulty method:")
    display(spectra.loc[fault_idxs_of_meth].reset_index(drop=True))

print("Top 5 SBFL scores:")
print("Op2:")
display(
    spectra.sort_values("Sop2", ascending=False).head(5).reset_index(drop=True)
)
print("Ochiai:")
display(
    spectra.sort_values("Sochiai", ascending=False)
    .head(5)
    .reset_index(drop=True)
)


fault meth: ('/home/workspace/src/projections/lcc.cpp', '_ZL9e_inverse5PJ_XYP8PJconsts')
fault idx: ('/home/workspace/src/projections/lcc.cpp', '_ZL9e_inverse5PJ_XYP8PJconsts', 110)
fault idx: ('/home/workspace/src/projections/lcc.cpp', '_ZL9e_inverse5PJ_XYP8PJconsts', 111)
SBFL score of faulty method:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts, 110)",7,1,31,0,0.820513,0.353553,6458.0,6458.0
1,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts, 111)",7,1,31,0,0.820513,0.353553,6458.0,6458.0


Top 5 SBFL scores:
Op2:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/projections/putp6.cpp, pj_putp6, 97)",0,1,38,0,1.0,1.0,3861.0,3861.0
1,"(/home/workspace/src/projections/cea.cpp, _ZL9e_forward5PJ_LPP8PJconsts, 22)",0,1,38,0,1.0,1.0,3861.0,3861.0
2,"(/home/workspace/src/projections/mbtfpq.cpp, _ZL9s_forward5PJ_LPP8PJconsts, 35)",0,1,38,0,1.0,1.0,3861.0,3861.0
3,"(/home/workspace/src/projections/mbtfpq.cpp, _ZL9s_inverse5PJ_XYP8PJconsts, 39)",0,1,38,0,1.0,1.0,3861.0,3861.0
4,"(/home/workspace/src/projections/mbtfpq.cpp, _ZL9s_inverse5PJ_XYP8PJconsts, 40)",0,1,38,0,1.0,1.0,3861.0,3861.0


Ochiai:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/projections/putp6.cpp, pj_putp6, 97)",0,1,38,0,1.0,1.0,3861.0,3861.0
1,"(/home/workspace/src/projections/cea.cpp, _ZL9e_forward5PJ_LPP8PJconsts, 22)",0,1,38,0,1.0,1.0,3861.0,3861.0
2,"(/home/workspace/src/projections/mbtfpq.cpp, _ZL9s_forward5PJ_LPP8PJconsts, 35)",0,1,38,0,1.0,1.0,3861.0,3861.0
3,"(/home/workspace/src/projections/mbtfpq.cpp, _ZL9s_inverse5PJ_XYP8PJconsts, 39)",0,1,38,0,1.0,1.0,3861.0,3861.0
4,"(/home/workspace/src/projections/mbtfpq.cpp, _ZL9s_inverse5PJ_XYP8PJconsts, 40)",0,1,38,0,1.0,1.0,3861.0,3861.0


# Method level SBFL

In [6]:
spectra["file"] = spectra.index.map(lambda x: x[0])
spectra["func"] = spectra.index.map(lambda x: x[1])
spectra["line"] = spectra.index.map(lambda x: x[2])
# (file, func) is meth
spectra["meth"] = spectra.apply(lambda row: (row.file, row.func), axis=1)
spectra.reset_index(drop=True, inplace=True)
display(spectra)

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth
0,71,4,0,34,1,-0.102564,0.0,31379.0,34245.0,/usr/include/c++/9/initializer_list,NaN,"(/usr/include/c++/9/initializer_list, nan)"
1,71,3,0,35,1,-0.076923,0.0,29575.0,34245.0,/usr/include/c++/9/initializer_list,_ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE4sizeEv,"(/usr/include/c++/9/initializer_list, _ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE4sizeEv)"
2,71,1,0,37,1,-0.025641,0.0,23891.0,34245.0,/usr/include/c++/9/initializer_list,_ZNKSt16initializer_listIN7dropbox6oxygen2nnISt10shared_ptrIN5osgeo4proj5datum9EllipsoidEEEEE4sizeEv,"(/usr/include/c++/9/initializer_list, _ZNKSt16initializer_listIN7dropbox6oxygen2nnISt10shared_ptrIN5osgeo4proj5datum9EllipsoidEEEEE4sizeEv)"
3,75,4,0,34,1,-0.102564,0.0,31379.0,34245.0,/usr/include/c++/9/initializer_list,_ZNKSt16initializer_listIN7dropbox6oxygen2nnISt10shared_ptrIN5osgeo4proj5datum9EllipsoidEEEEE4sizeEv,"(/usr/include/c++/9/initializer_list, _ZNKSt16initializer_listIN7dropbox6oxygen2nnISt10shared_ptrIN5osgeo4proj5datum9EllipsoidEEEEE4sizeEv)"
4,75,3,0,35,1,-0.076923,0.0,29575.0,34245.0,/usr/include/c++/9/initializer_list,_ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE5beginEv,"(/usr/include/c++/9/initializer_list, _ZNKSt16initializer_listIN5osgeo4proj2io16AuthorityFactory10ObjectTypeEE5beginEv)"
...,...,...,...,...,...,...,...,...,...,...,...,...
34240,91,0,1,38,0,1.000000,1.0,3861.0,3861.0,/home/workspace/src/projections/putp6.cpp,pj_putp6,"(/home/workspace/src/projections/putp6.cpp, pj_putp6)"
34241,93,0,1,38,0,1.000000,1.0,3861.0,3861.0,/home/workspace/src/projections/putp6.cpp,pj_putp6,"(/home/workspace/src/projections/putp6.cpp, pj_putp6)"
34242,94,0,1,38,0,1.000000,1.0,3861.0,3861.0,/home/workspace/src/projections/putp6.cpp,pj_putp6,"(/home/workspace/src/projections/putp6.cpp, pj_putp6)"
34243,95,0,1,38,0,1.000000,1.0,3861.0,3861.0,/home/workspace/src/projections/putp6.cpp,pj_putp6,"(/home/workspace/src/projections/putp6.cpp, pj_putp6)"


## Max aggregation

In [7]:
print("[Op2]")
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_op2 = spectra_max_op2[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sop2"]
]
spectra_max_op2["Rop2"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_op2.loc[spectra_max_op2.meth.isin(fault_meths)].reset_index(
        drop=True
    )
)
print("Top 20 methods:")
display(spectra_max_op2.head(20).reset_index(drop=True))
print()
print("[Ochiai]")
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_ochiai = spectra_max_ochiai[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sochiai"]
]
spectra_max_ochiai["Rochiai"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_ochiai.loc[
        spectra_max_ochiai.meth.isin(fault_meths)
    ].reset_index(drop=True)
)
print("Top 20 methods:")
display(spectra_max_ochiai.head(20).reset_index(drop=True))


[Op2]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",123,0,1,38,0,1.0,307.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/src/projections/wink2.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",56,0,1,38,0,1.0,307.0
1,"(/home/workspace/src/projections/wink1.cpp, _ZL9s_inverse5PJ_XYP8PJconsts)",46,0,1,38,0,1.0,307.0
2,"(/home/workspace/src/projections/wink1.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",23,0,1,38,0,1.0,307.0
3,"(/home/workspace/src/projections/wag7.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",22,0,1,38,0,1.0,307.0
4,"(/home/workspace/src/projections/wag3.cpp, _ZL9s_inverse5PJ_XYP8PJconsts)",32,0,1,38,0,1.0,307.0
5,"(/home/workspace/src/projections/wag3.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",24,0,1,38,0,1.0,307.0
6,"(/home/workspace/src/projections/wag2.cpp, _ZL9s_inverse5PJ_XYP8PJconsts)",30,0,1,38,0,1.0,307.0
7,"(/home/workspace/src/projections/wag2.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",21,0,1,38,0,1.0,307.0
8,"(/home/workspace/src/projections/vandg4.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",55,0,1,38,0,1.0,307.0
9,"(/home/workspace/src/projections/vandg2.cpp, pj_vandg2)",76,0,1,38,0,1.0,307.0



[Ochiai]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",123,0,1,38,0,1.0,307.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/src/projections/wink2.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",56,0,1,38,0,1.0,307.0
1,"(/home/workspace/src/projections/wink1.cpp, _ZL9s_inverse5PJ_XYP8PJconsts)",46,0,1,38,0,1.0,307.0
2,"(/home/workspace/src/projections/wink1.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",23,0,1,38,0,1.0,307.0
3,"(/home/workspace/src/projections/wag7.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",22,0,1,38,0,1.0,307.0
4,"(/home/workspace/src/projections/wag3.cpp, _ZL9s_inverse5PJ_XYP8PJconsts)",32,0,1,38,0,1.0,307.0
5,"(/home/workspace/src/projections/wag3.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",24,0,1,38,0,1.0,307.0
6,"(/home/workspace/src/projections/wag2.cpp, _ZL9s_inverse5PJ_XYP8PJconsts)",30,0,1,38,0,1.0,307.0
7,"(/home/workspace/src/projections/wag2.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",21,0,1,38,0,1.0,307.0
8,"(/home/workspace/src/projections/vandg4.cpp, _ZL9s_forward5PJ_LPP8PJconsts)",55,0,1,38,0,1.0,307.0
9,"(/home/workspace/src/projections/vandg2.cpp, pj_vandg2)",76,0,1,38,0,1.0,307.0


## Line2Method voting (weight: sbfl score / dense rank)

In [8]:
print("[Op2]")
ballot_op2 = {}
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2["Rop2D"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="dense"
)
spectra_max_op2["vote"] = spectra_max_op2.Sop2 / spectra_max_op2.Rop2D
for row in spectra_max_op2.itertuples():
    if row.meth not in ballot_op2:
        ballot_op2[row.meth] = row.vote
    else:
        ballot_op2[row.meth] += row.vote
L2M_op2 = pd.DataFrame(
    {"meth": list(ballot_op2.keys()), "vote": list(ballot_op2.values())}
)
L2M_op2["rank"] = L2M_op2.vote.rank(ascending=False, method="max")
L2M_op2.sort_values("rank", inplace=True)
L2M_op2.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(L2M_op2.loc[L2M_op2.meth == fault_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == fault_meth].reset_index(
            drop=True
        )
    )
print("Top 5 methods:")
display(L2M_op2.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_op2.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_op2.loc[L2M_op2.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == top_meth].reset_index(
            drop=True
        )
    )
print()
print("[Ochiai]")
ballot_ochiai = {}
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai["RochiaiD"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="dense"
)
spectra_max_ochiai["vote"] = (
    spectra_max_ochiai.Sochiai / spectra_max_ochiai.RochiaiD
)
for row in spectra_max_ochiai.itertuples():
    if row.meth not in ballot_ochiai:
        ballot_ochiai[row.meth] = row.vote
    else:
        ballot_ochiai[row.meth] += row.vote
L2M_ochiai = pd.DataFrame(
    {"meth": list(ballot_ochiai.keys()), "vote": list(ballot_ochiai.values())}
)
L2M_ochiai["rank"] = L2M_ochiai.vote.rank(ascending=False, method="max")
L2M_ochiai.sort_values("rank", inplace=True)
L2M_ochiai.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(
        L2M_ochiai.loc[L2M_ochiai.meth == fault_meth].reset_index(drop=True)
    )
    print("Voters:")
    display(
        spectra_max_ochiai.loc[
            spectra_max_ochiai.meth == fault_meth
        ].reset_index(drop=True)
    )
print("Top 5 methods:")
display(L2M_ochiai.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_ochiai.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_ochiai.loc[L2M_ochiai.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_ochiai.loc[spectra_max_ochiai.meth == top_meth].reset_index(
            drop=True
        )
    )


[Op2]
Faulty method:


,meth,vote,rank
0,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",12.221929,130.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,Rop2D,vote
0,123,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/lcc.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
1,122,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/lcc.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
2,121,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/lcc.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
3,120,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/lcc.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
4,119,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/lcc.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
5,118,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/lcc.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
6,117,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/lcc.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
7,70,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/lcc.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
8,76,5,1,33,0,0.871795,0.408248,6292.0,6292.0,/home/workspace/src/projections/lcc.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",6.0,0.145299
9,71,5,1,33,0,0.871795,0.408248,6292.0,6292.0,/home/workspace/src/projections/lcc.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/lcc.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",6.0,0.145299


Top 5 methods:


...

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,157,0,1,38,0,1.0,1.0,3861.0,3861.0,/home/workspace/src/projections/rouss.cpp,_ZL10destructorP8PJconstsi,"(/home/workspace/src/projections/rouss.cpp, _ZL10destructorP8PJconstsi)",1.0,1.0
1,155,0,1,38,0,1.0,1.0,3861.0,3861.0,/home/workspace/src/projections/rouss.cpp,_ZL10destructorP8PJconstsi,"(/home/workspace/src/projections/rouss.cpp, _ZL10destructorP8PJconstsi)",1.0,1.0
2,154,0,1,38,0,1.0,1.0,3861.0,3861.0,/home/workspace/src/projections/rouss.cpp,_ZL10destructorP8PJconstsi,"(/home/workspace/src/projections/rouss.cpp, _ZL10destructorP8PJconstsi)",1.0,1.0
3,153,0,1,38,0,1.0,1.0,3861.0,3861.0,/home/workspace/src/projections/rouss.cpp,_ZL10destructorP8PJconstsi,"(/home/workspace/src/projections/rouss.cpp, _ZL10destructorP8PJconstsi)",1.0,1.0
4,151,0,1,38,0,1.0,1.0,3861.0,3861.0,/home/workspace/src/projections/rouss.cpp,_ZL10destructorP8PJconstsi,"(/home/workspace/src/projections/rouss.cpp, _ZL10destructorP8PJconstsi)",1.0,1.0
5,150,0,1,38,0,1.0,1.0,3861.0,3861.0,/home/workspace/src/projections/rouss.cpp,_ZL10destructorP8PJconstsi,"(/home/workspace/src/projections/rouss.cpp, _ZL10destructorP8PJconstsi)",1.0,1.0
6,149,0,1,38,0,1.0,1.0,3861.0,3861.0,/home/workspace/src/projections/rouss.cpp,_ZL10destructorP8PJconstsi,"(/home/workspace/src/projections/rouss.cpp, _ZL10destructorP8PJconstsi)",1.0,1.0
7,148,0,1,38,0,1.0,1.0,3861.0,3861.0,/home/workspace/src/projections/rouss.cpp,_ZL10destructorP8PJconstsi,"(/home/workspace/src/projections/rouss.cpp, _ZL10destructorP8PJconstsi)",1.0,1.0
8,147,0,1,38,0,1.0,1.0,3861.0,3861.0,/home/workspace/src/projections/rouss.cpp,_ZL10destructorP8PJconstsi,"(/home/workspace/src/projections/rouss.cpp, _ZL10destructorP8PJconstsi)",1.0,1.0
9,146,0,1,38,0,1.0,1.0,3861.0,3861.0,/home/workspace/src/projections/rouss.cpp,_ZL10destructorP8PJconstsi,"(/home/workspace/src/projections/rouss.cpp, _ZL10destructorP8PJconstsi)",1.0,1.0


Top 5:


,meth,vote,rank
0,"(/home/workspace/src/projections/labrd.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",49.838835,5.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,112,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/labrd.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/labrd.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
1,100,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/labrd.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/labrd.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
2,99,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/labrd.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/labrd.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
3,98,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/labrd.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/labrd.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
4,97,0,1,38,0,1.000000,1.000000,3861.0,3861.0,/home/workspace/src/projections/labrd.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/labrd.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,111,1,1,37,0,0.974359,0.707107,5470.0,5470.0,/home/workspace/src/projections/labrd.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/labrd.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",2.0,0.353553
62,109,1,1,37,0,0.974359,0.707107,5470.0,5470.0,/home/workspace/src/projections/labrd.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/labrd.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",2.0,0.353553
63,107,1,1,37,0,0.974359,0.707107,5470.0,5470.0,/home/workspace/src/projections/labrd.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/labrd.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",2.0,0.353553
64,106,1,1,37,0,0.974359,0.707107,5470.0,5470.0,/home/workspace/src/projections/labrd.cpp,_ZL9e_inverse5PJ_XYP8PJconsts,"(/home/workspace/src/projections/labrd.cpp, _ZL9e_inverse5PJ_XYP8PJconsts)",2.0,0.353553


# 감상

단순히 passing test에 의해서 한 번도 실행되지 않고 failing test에 의해서 실행되는 method가 너무 많다 (sbfl max agg top max rank: 269)